கீழ்கண்ட நோட்புக்குகளை இயக்குவதற்காக, நீங்கள் இதுவரை செய்யவில்லை என்றால், `text-embedding-ada-002` அடிப்படை மாடலை பயன்படுத்தும் ஒரு மாடலை பிரயோகத்தில் அமைக்க வேண்டும் மற்றும் `.env` கோப்பில் பிரயோகத்தின் பெயரை `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` ஆக அமைக்க வேண்டும்.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

அடுத்ததாக, நாங்கள் Embedding Index ஐ Pandas Dataframe இல் ஏற்ற உள்ளோம். Embedding Index `embedding_index_3m.json` என்ற JSON கோப்பில் சேமிக்கப்பட்டுள்ளது. Embedding Index, அக்டோபர் 2023 இறுதிவரை உள்ள YouTube உரைகளுக்கான Embeddings ஐ கொண்டுள்ளது.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

அடுத்து, `get_videos` என்ற ஒரு செயல்பாட்டை உருவாக்கப் போகிறோம், இது Embedding Index-இல் கேள்விக்கான தேடலைச் செய்யும். இந்த செயல்பாடு கேள்வியுடன் மிகவும் தொடர்புடைய முதல் 5 வீடியோக்களை திருப்பி அனுப்பும். இந்த செயல்பாடு கீழ்க்கண்டவாறு செயல்படுகிறது:

1. முதலில், Embedding Index-இன் ஒரு நகல் உருவாக்கப்படுகிறது.
2. அடுத்ததாக, OpenAI Embedding API-யை பயன்படுத்தி கேள்விக்கான Embedding கணக்கிடப்படுகிறது.
3. பின்னர், Embedding Index-இல் `similarity` என்ற புதிய நெடுவரிசை உருவாக்கப்படுகிறது. `similarity` நெடுவரிசை, கேள்விக்கான Embedding மற்றும் ஒவ்வொரு வீடியோ பகுதியின் Embedding இடையேயான கோசைன் ஒற்றுமையை கொண்டுள்ளது.
4. அடுத்ததாக, Embedding Index `similarity` நெடுவரிசை மூலம் வடிகட்டப்படுகிறது. கோசைன் ஒற்றுமை 0.75 அல்லது அதற்கு மேல் உள்ள வீடியோக்களை மட்டுமே உள்ளடக்குவதற்காக Embedding Index வடிகட்டப்படுகிறது.
5. இறுதியாக, Embedding Index `similarity` நெடுவரிசை மூலம் வரிசைப்படுத்தப்படுகிறது, மேலும் முதல் 5 வீடியோக்கள் திருப்பி அனுப்பப்படுகின்றன.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

இந்த செயல்பாடு மிகவும் எளிமையானது, இது தேடல் கேள்வியின் முடிவுகளை அச்சிடுகிறது.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. முதலில், Embedding Index ஒரு Pandas Dataframe-இல் ஏற்றப்படுகிறது.
2. அடுத்ததாக, பயனர் ஒரு கேள்வியை உள்ளிடுமாறு கேட்டுக்கொள்ளப்படுகிறார்.
3. பின்னர், `get_videos` செயல்பாடு கேள்விக்கான Embedding Index-ஐ தேட அழைக்கப்படுகிறது.
4. இறுதியாக, `display_results` செயல்பாடு பயனருக்கு முடிவுகளை காட்ட அழைக்கப்படுகிறது.
5. அதன் பிறகு, பயனர் மற்றொரு கேள்வியை உள்ளிடுமாறு கேட்டுக்கொள்ளப்படுகிறார். பயனர் `exit` என உள்ளிடும் வரை இந்த செயல்முறை தொடரும்.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.ta.png)

உங்களுக்கு ஒரு கேள்வியை உள்ளிடுமாறு கேட்டுக்கொள்ளப்படும். ஒரு கேள்வியை உள்ளிட்டு Enter அழுத்தவும். பயன்பாடு அந்த கேள்விக்கு தொடர்புடைய வீடியோக்களின் பட்டியலை திருப்பும். மேலும், கேள்விக்கு பதில் உள்ள வீடியோவின் இடத்திற்கான இணைப்பையும் திருப்பும்.

இங்கே சில கேள்விகளை முயற்சிக்கலாம்:

- Azure Machine Learning என்றால் என்ன?
- Convolutional neural networks எப்படி செயல்படுகின்றன?
- Neural network என்றால் என்ன?
- Azure Machine Learning-இன் மூலம் Jupyter Notebooks-ஐ பயன்படுத்த முடியுமா?
- ONNX என்றால் என்ன?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. எங்கள் தரச்செயல்முறைகளுக்கு முக்கியத்துவம் அளிக்கின்றோம், ஆனால் தானியக்க மொழிபெயர்ப்புகளில் பிழைகள் அல்லது துல்லியமின்மைகள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
